# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [2]:
from trading_bot.agent import Agent
stock = 'BPCL'
model_name = '{}_t-dqn_40'.format(stock)
test_stock = 'data/Nifty50/Split/{}_2019.csv'.format(stock)
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

Using TensorFlow backend.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



### 2. Load test data

In [3]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# df = df.iloc[:55]
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-01,321.005829
1,2019-01-02,312.919495
2,2019-01-03,308.155090
3,2019-01-04,314.230804
4,2019-01-07,310.821381


### 3. Running Eval

In [4]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-09-28 19:34:13 Tornado root[8660] DEBUG switching to TensorFlow for CPU
2020-09-28 19:34:13 Tornado root[8660] DEBUG Buy at: $321.01
2020-09-28 19:34:13 Tornado root[8660] DEBUG Buy at: $312.92
2020-09-28 19:34:13 Tornado root[8660] DEBUG Buy at: $308.16
2020-09-28 19:34:13 Tornado root[8660] DEBUG Buy at: $310.82
2020-09-28 19:34:13 Tornado root[8660] DEBUG Buy at: $307.67
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $293.38
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $294.87
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $292.81
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $298.71 | Position: -$22.29
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $303.57 | Position: -$9.35
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $310.52 | Position: +$2.36
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $309.64
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $307.32
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $309.29 | Posit

2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $387.34
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $360.27 | Position: +$63.52
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $349.61 | Position: +$37.35
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $362.26 | Position: +$44.28
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $360.13
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $382.89
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $437.35 | Position: +$112.82
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $441.05
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $455.13 | Position: +$118.32
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $445.41 | Position: +$107.70
2020-09-28 19:34:14 Tornado root[8660] DEBUG Buy at: $445.70
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $468.26 | Position: +$123.58
2020-09-28 19:34:14 Tornado root[8660] DEBUG Sell at: $504.00 | Position: +$141.83
2020-09-28 19:34:14 Tornado root[8

### 4. Visualize

In [5]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [6]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [7]:
cap = [0]
inv = 0
ret = 0
b = 0
for i in range(len(df)):
    if df.iloc[i]['action']=='BUY':
        cap.append(cap[-1]+df.iloc[i]['actual'])
        inv+=df.iloc[i]['actual']
        b+=1
    if df.iloc[i]['action']=='SELL' and b>0:
        cap.append(cap[-1]-df.iloc[i]['actual'])
        ret += df.iloc[i]['actual']
        b-=1

In [8]:
req_cap = max(cap)
req_cap

3496.219881

In [9]:
prof = ret+(df['action'].value_counts()['BUY']-df['action'].value_counts()['SELL'])*df.iloc[-1]['actual']-inv

In [10]:
(prof/ req_cap)*100

35.094141494586076

In [11]:
df['action'].value_counts()

BUY     89
SELL    82
HOLD    70
Name: action, dtype: int64

In [105]:
list(full['action'].iloc[:len(df)]) == list(df['action'])

True

In [88]:
full

,date,actual,position,action
0,2018-01-01,1715.731567,1715.731567,HOLD
1,2018-01-02,1714.737305,1715.731567,BUY
2,2018-01-03,1705.440918,1714.737305,SELL
3,2018-01-04,1743.918579,1705.440918,BUY
4,2018-01-05,1804.070801,1743.918579,HOLD
5,2018-01-08,1826.292358,1804.070801,SELL
6,2018-01-09,1809.042114,1826.292358,HOLD
7,2018-01-10,1791.841553,1809.042114,HOLD
8,2018-01-11,1770.117065,1791.841553,BUY
9,2018-01-12,1759.926025,1770.117065,BUY


In [ ]:
import os
import multiprocessing
import time

from multiprocessing import Process

def f(name):
    time.sleep(5)
    print('hello', name)

f('mat')
f('bob')

ps = []
ps.append(Process(target=f, args=('bob',)))
ps.append(Process(target=f, args=('mat',)))
ps.append(Process(target=f, args=('jay',)))
for i in ps:
    i.start()

os.system("python3 train.py 'data/India/IndusINDVal.csv' 'data/India/IndusINDVal.csv' --strategy='t-dqn' --model-name='IndusInd'")